# Downloads

In [ ]:
#!pip install sentencepiece

# Imports

In [13]:
import os
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace 
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

# Setting up LLM

## Importing the .Env variable

In [3]:
HF_TOKEN=os.environ.get('HF_TOKEN')
huggingface_repo_id="mistralai/Mistral-7B-Instruct-v0.3"

keeping the `temperature` low makes is accurate and very concise in the answers

In [4]:
def load_llm(huggingface_repo_id):
    llm = HuggingFaceEndpoint(  
        repo_id=huggingface_repo_id, 
        temperature=0.5,
        huggingfacehub_api_token=HF_TOKEN,
        task='conversational'
    )
    return llm


# Connecting to LLM and creating a chain

## Setting the prompt template

In [5]:
custom_prompt_template = """
Use the pieces of information provided in the context to answer user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Don't provide anything out of the given context

Context: {context}
Question: {question}

Start the answer directly. No small talk please.
"""

In [6]:
def set_custom_prompt(custom_prompt_template):
    prompt= PromptTemplate(template=custom_prompt_template,
                          input_variables=["context", "question"])
    return prompt

## Loading the FAISS Database

In [7]:
embedding_model= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

In [8]:
FAISS_DB_PATH = 'FAISS Database/'

Set the `allow_dangerous_deserialization` parameter to `True` becuase we trust the `FAISS` database
and its contents inside it because we made it

In [9]:
db = FAISS.load_local(FAISS_DB_PATH,
                      embedding_model,
                      allow_dangerous_deserialization=True)

## Creating the QA Chain

In [14]:
chat_model = ChatHuggingFace(llm=load_llm(huggingface_repo_id))
qa_chain=RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={'k':3}),
    return_source_documents=True,
    chain_type_kwargs={'prompt':set_custom_prompt(custom_prompt_template)}
)

## Creating user input

In [17]:
user_query=input("Ask WLF ")
response=qa_chain.invoke({'query': user_query})
print("Result : ", response["result"])
print("Source Documents: ", response["source_documents"])

Ask WLF  when is gta 6 gettng released


Result :   I'm sorry for any confusion, but the context provided doesn't contain any information about the release date of GTA 6. The information in the context pertains to a video game walkthrough, not real-world events or game release dates.
Source Documents:  [Document(id='8e81b396-5d92-43a1-8cca-b818265e35ab', metadata={}, page_content='already picked up the page, but it will be on the filing cabinet in front of the coffee machine. The note will talk about the code being the “Big Win”. From here, just turn around to the bulletin board and there will be a “TERRA LOTTO” ticket. The code will be the circled numbers. Inside, you will find Ammo and a new gun, the Hunting Pistol .'), Document(id='e2477a0d-f840-48c1-8f86-23b0c8376e88', metadata={}, page_content="Vault Code Once inside the vault you will find a dead body with a shotgun on it, and a note about the guy getting trapped inside the vault. Put away the note after reading it, and enjoy your new weapon! (Note: it is still possible